In [1]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm_notebook # check the progressbar in the python. 
import glob # check the file name in fold. 

In [2]:
import pyspark

import findspark
findspark.init()
findspark.find()

'C:\\spark-2.4.4-bin-hadoop2.7\\spark-2.4.4-bin-hadoop2.7'

In [3]:
# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'

In [4]:
read_file_lst = glob.glob(path + '*')
read_file_lst[0:2]

['C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final\\fin_test_accounts.csv',
 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final\\labeled_accounts.csv']

In [5]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [7]:
from pyspark.sql import SQLContext
from pyspark import SparkFiles
from pyspark.sql.functions import lit

sqlContext = SQLContext(sc)
for i in tqdm_notebook(read_file_lst):
    # sqlContext.read.csv() : path를 C:Users/..의 형식으로 맞춰줘야함. (초기값이 spark설치된 곳으로 지정되어있음)
    df_temp = sqlContext.read.csv(SparkFiles.get(i), header=True, inferSchema= True) # read.csv 
    
    # transformation으로 추가하려면, df = df.withColumn("age_square", col("age")**2)
    df_temp = df_temp.withColumn("Date", lit(i[-8:-4])) # column 추가, lit명령어가 value를 추가하는 방법. 
    
    if i == read_file_lst[0]:
        df_total = df_temp
    else:
        df_total = unionAll([df_total, df_temp]) # row-wise 결합
        
    del df_temp 
    gc.collect()

AnalysisException: "cannot resolve '`account`' given input columns: [etc_num14, worldnum, actor_account, etc_num15, etc_num11, etc_num1, item_uid, Date, etc_num4, etc_num2, etc_str1, actor, etc_num10, etc_num9, etc_num7, log_id, etc_str3, etc_num3, target_account, etc_num6, etc_num5, etc_num8, big_log_id, etc_num12, log_date, target, etc_str2, etc_num13];;\n'Project ['account, Date#101]\n+- Project [log_date#47, big_log_id#48, log_id#49, actor#50, actor_account#51, target#52, target_account#53, worldnum#54L, etc_str1#55, etc_str2#56, etc_str3#57, etc_num1#58L, etc_num2#59L, etc_num3#60L, etc_num4#61, etc_num5#62L, etc_num6#63L, etc_num7#64L, etc_num8#65L, etc_num9#66L, etc_num10#67, etc_num11#68L, etc_num12#69L, etc_num13#70L, ... 4 more fields]\n   +- Relation[log_date#47,big_log_id#48,log_id#49,actor#50,actor_account#51,target#52,target_account#53,worldnum#54L,etc_str1#55,etc_str2#56,etc_str3#57,etc_num1#58L,etc_num2#59L,etc_num3#60L,etc_num4#61,etc_num5#62L,etc_num6#63L,etc_num7#64L

In [ ]:
import pyspark.sql.functions as f
# userItem=df.groupby('userId').agg(f.expr('count(distinct item)').alias('n_item'))
# df_total.groupBy("log_id").agg(f.expr('count(distinct item)').alias('log_id'))
# df_total.select("actor_account").distinct().count()

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "fin_test_accounts.csv")
del train_label['class']

### Feature Engineering

Group activities features
- Avg_PartyTime
- GuildAct_count (계산 못하겠음)
- GuildJoin_count

In [ ]:
df_total = df_total.withColumnRenamed("actor_account", "account")

In [ ]:
# Avg_PartyTime
df_total_agg = df_total.filter(df_total['log_id'] == 127).select('account','etc_num12').toPandas()
df_total_agg = df_total_agg.groupby(['account'])['etc_num12'].agg({'sum','mean'}).reset_index().rename(columns={'sum':'Sum_PartyTime', 'mean':'Avg_PartyTime'})
train_label = train_label.merge(df_total_agg, on = 'account', how='left')
test_label = test_label.merge(df_total_agg, on = 'account', how='left')

In [ ]:
#GuildJoin_count
df_total_agg = df_total.filter(df_total['log_id'] == 605).select('account').toPandas()
df_total_agg = df_total_agg.groupby(['account'])['account'].agg({'count'}).reset_index().rename(columns={'count':'GuildJoin_count'})
train_label = train_label.merge(df_total_agg, on = 'account', how='left')
test_label = test_label.merge(df_total_agg, on = 'account', how='left')

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features2/'
train_label.to_csv(outputpath + "train_group.csv", index=False)
test_label.to_csv(outputpath + "test_group.csv", index=False)

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "fin_test_accounts.csv")
del train_label['class']

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account').toPandas()
df_total_agg = df_total_agg.groupby('account')['account'].agg({'count'}).reset_index().rename(columns={'count':'login_count'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account','date').toPandas()
df_total_agg = df_total_agg.groupby('account')['date'].agg({'nunique'}).reset_index().rename(columns={'nunique':'Day_unique_count'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account','etc_str1').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_str1'].agg({'nunique'}).reset_index().rename(columns={'nunique':'ip_nunique'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 103).select('account','etc_num2').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num2'].agg({'max'}).reset_index().rename(columns={'max':'max_level'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 104).select('account','etc_num7').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num7'].agg({'mean', 'sum'}).reset_index().rename(columns={'mean':'mean_playtime', 'sum':'sum_playtime'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 104).select('account','etc_num1').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num1'].agg({'mean', 'sum'}).reset_index().rename(columns={'sum':'sum_money'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
df_total_agg = df_total.filter(df_total['log_id'] == 116).select('account','etc_num7','etc_num8','etc_num9','etc_num10').toPandas()
df_total_agg = df_total_agg.groupby('account')[['etc_num7','etc_num8','etc_num9','etc_num10']].agg({'mean'}).reset_index()
df_total_agg.columns = ['account', 'captcha_count_mean', 'captcha_ban_time_mean', 'captcha_prior_time_mean', 'captcha_occur_time_mean']

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features2/'
train_label.to_csv(outputpath + "train_player.csv", index=False)
test_label.to_csv(outputpath + "test_player.csv", index=False)

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "fin_test_accounts.csv")
del train_label['class']

In [ ]:
def social_network_features(train, test, df, log_id, name):
    # In-degree
    df_agg1 = df.filter(df['log_id'] == log_id).groupBy(['account']).agg({'account':'count'}).withColumnRenamed("count(account)", "{}_in_deg".format(name))
    # Out-degree
    df_agg2 = df.filter(df['log_id'] == log_id).groupBy(['target_account']).agg({'target_account':'count'}).withColumnRenamed("count(target_account)", "{}_out_deg".format(name))
    df_agg2 = df_agg2.withColumnRenamed("target_account", "account")
    
    df_agg1 = df_agg1.select('account',"{}_in_deg".format(name)).toPandas()
    df_agg2 = df_agg2.select('account',"{}_out_deg".format(name)).toPandas()
    # Python code로 짜야하는데 어려움 ㅜㅜ... 
    # package는 apache기준. pyspark의 package는 pagerank랑 Component_ID만 있음. 
    # cc, between, closeness, Eigenvector, Eccentricity, Authority, Hub, Pagerank 
    
    #train = train.join(df_agg1, ['account'], how='left')
    test = pd.merge(test, df_agg1, on = 'account', how='left')

    #train = train.join(df_agg2, ['account'], how='left')
    test = pd.merge(test, df_agg2, on = 'account', how='left')
    print("end..")
    return test

In [ ]:
# party 
test_label = social_network_features(train_label, test_label, df_total, log_id = 126, name = 'p')

# Friend 
test_label = social_network_features(train_label, test_label, df_total, log_id = 134, name = 'f')

# Dual
test_label = social_network_features(train_label, test_label, df_total, log_id = 158, name = 'd')

# Mail 
test_label = social_network_features(train_label, test_label, df_total, log_id = 229, name = 'm')

# Trade1
test_label = social_network_features(train_label, test_label, df_total, log_id = 210, name = 't1')

# Trade2
test_label = social_network_features(train_label, test_label, df_total, log_id = 219, name = 't2')

# Private Shop
test_label = social_network_features(train_label, test_label, df_total, log_id = 247, name = 'p_s')

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features2/'
train_label.to_csv(outputpath + "train_social.csv", index=False)
test_label.to_csv(outputpath + "test_social.csv", index=False)

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "fin_test_accounts.csv")
del train_label['class']

In [ ]:
def action_features(train, test, df, name, count, ratio, per_day_count, log_id, etc_num):
    df_agg = df.filter(df['log_id'] == log_id).select('account', 'date').toPandas()
    df_agg = df_agg.groupby(['account', 'date'])['account'].agg({'count'}).reset_index()
    df_agg = df_agg.groupby(['account'])['count'].agg({'mean', 'sum', 'std'}).reset_index().rename(columns={'mean':'{}_per_day_count'.format(name), 'sum':'{}_sum_count'.format(name), 'std':'{}_std_per_day_count'.format(name)})
    # train = train.merge(df_agg, on='account' , how='left')
    test = test.merge(df_agg, on='account' , how='left')
        
    df_agg = df.filter(df['log_id'] == log_id).select('account').toPandas()
    df_agg = df_agg.groupby(['account'])['account'].agg({'count'}).reset_index().rename(columns={'count':"{}_count".format(name)})
    # train = train.merge(df_agg, on='account' , how='left')
    test = test.merge(df_agg, on='account' , how='left')

    print("Processing end...")
    return train, test

In [ ]:
# Exp_repair information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Exp_repair', log_id = 148 , etc_num = 'account', count = True, ratio = False, per_day_count = True )

# Use_portal False
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Use_portal', log_id = 151 , etc_num = 'account', count = True, ratio = False, per_day_count = True )

# Killed_bypc information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Killed_bypc', log_id = 137 , etc_num = 'account', count = True, ratio = False, per_day_count = True )

# Killed_bynpc information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Killed_bynpc', log_id = 138 , etc_num = 'account', count = True, ratio = False, per_day_count = True )

# Teleport information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Teleport', log_id = 142 , etc_num = 'account', count = True, ratio = False, per_day_count = True )

# Reborn information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'Reborn', log_id = 145 , etc_num = 'account', count = True, ratio = False, per_day_count = True )

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features2/'
train_label.to_csv(outputpath + "train_action2.csv", index=False)
test_label.to_csv(outputpath + "test_action2.csv", index=False)

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "fin_test_accounts.csv")
del train_label['class']

In [ ]:
def action_features(train, test, df, name, count, ratio, per_day_count, log_id, etc_num):
    if per_day_count == True:
        #print("Processing per_day_count features...")
        df_agg = df.filter(df['log_id'] == log_id).select('account', 'date').toPandas()
        df_agg = df_agg.groupby(['account', 'date'])['account'].agg({'count'}).reset_index()
        df_agg = df_agg.groupby(['account'])['count'].agg({'mean', 'sum'}).reset_index()
        df_agg = df_agg.rename(columns={'mean':'{}_per_day_count'.format(name), 'sum':'{}_sum_count'.format(name), 'std':'{}_std_per_day_count'.format(name)})
        # train = train.merge(df_agg, on='account' , how='left')
        test = test.merge(df_agg, on='account' , how='left')
        
    if count == True:
        #print("Processing count features...")
        df_agg = df.filter(df['log_id'] == log_id).select('account').toPandas()
        df_agg = df_agg.groupby(['account'])['account'].agg({'count'}).reset_index().rename(columns={'count':"{}_count".format(name)})
        # train = train.merge(df_agg, on='account' , how='left')
        test = test.merge(df_agg, on='account' , how='left')
        #if ratio == True:
        #    # "{}_count".format(name)
        #    # Count (총) / Sum (총)
        #    # etc_num을 Sum으로 나누면 Ratio가 됨. 
        #    df_agg = df.filter(df['log_id'] == log_id).groupBy('account').agg({'account_account':'count', '{}'.format(etc_num):'sum'}).withColumnRenamed("count(account_account)", "{}_count".format(name)).withColumnRenamed("sum({})".format(etc_num), "{}_sum".format(name))
        #    df = df.join(df_agg, ['account'], how='left')
        #    df = df.withColumn('{}_ratio'.format(name), df['etc_num']/df["{}_sum".format(name)]).drop("{}_sum".format(name))
    print("Processing end...")
    return train, test

In [ ]:
#Feature name : donepoll
df_total_agg = df_total.filter(df_total['log_id'] == 152).select('account').toPandas()
df_total_agg = df_total_agg.groupby('account')['account'].agg({'count'}).reset_index().rename(columns={'count':'count_donepoll'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
# Quest_Try
df_total_agg = df_total.filter(df_total['log_id'] == 501).select('account').toPandas()
df_total_agg = df_total_agg.groupby('account')['account'].agg({'count'}).reset_index().rename(columns={'count':'quest_try_count'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
# Quest_Success
df_total_agg = df_total.filter(df_total['log_id'] == 503).select('account').toPandas()
df_total_agg = df_total_agg.groupby('account')['account'].agg({'count'}).reset_index().rename(columns={'count':'quest_success_count'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
# GatherLimitUp
df_total_agg = df_total.filter(df_total['log_id'] == 416).select('account','etc_num8').toPandas()
df_total_agg = df_total_agg.groupby('account')['etc_num8'].agg({'count', 'sum'}).reset_index().rename(columns={'sum':'sum_gather', 'count':'count_gather'})

train_label = train_label.merge(df_total_agg, on='account' , how='left')
test_label = test_label.merge(df_total_agg, on='account' , how='left')

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features2/'
train_label.to_csv(outputpath + "train_action3.csv", index=False)
test_label.to_csv(outputpath + "test_action3.csv", index=False)

In [ ]:
i = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/Challenge19_GameBot_Final/'
train_label = pd.read_csv(i + "labeled_accounts.csv")
test_label = pd.read_csv(i + "fin_test_accounts.csv")
del train_label['class']

In [ ]:
def action_features(train, test, df, name, count, ratio, per_day_count, log_id, etc_num):
    df_agg = df.filter(df['log_id'] == log_id).select('account', 'date').toPandas()
    df_agg = df_agg.groupby(['account', 'date'])['account'].agg({'count'}).reset_index()
    df_agg = df_agg.groupby(['account'])['count'].agg({'mean', 'sum'}).reset_index()
    df_agg = df_agg.rename(columns={'mean':'{}_per_day_count'.format(name), 'sum':'{}_sum_count'.format(name), 'std':'{}_std_per_day_count'.format(name)})
    # train = train.merge(df_agg, on='account' , how='left')
    test = test.merge(df_agg, on='account' , how='left')
        
    df_agg = df.filter(df['log_id'] == log_id).select('account').toPandas()
    df_agg = df_agg.groupby(['account'])['account'].agg({'count'}).reset_index().rename(columns={'count':"{}_count".format(name)})
    # train = train.merge(df_agg, on='account' , how='left')
    test = test.merge(df_agg, on='account' , how='left')

    print("Processing end...")
    return train, test

In [ ]:
# sit information
# train_label, test_label = action_features(train_label, test_label, df_total, name = 'sit', log_id = 118 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# Exp_get information
# train_label, test_label = action_features(train_label, test_label, df_total, name = 'Exp_get', log_id = 143 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# item_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'item_get', log_id = 225 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# money_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'money_get', log_id = 187 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

# abyss_get information
train_label, test_label = action_features(train_label, test_label, df_total, name = 'abyss_get', log_id = 156 , etc_num = 'account', count = True, ratio = True, per_day_count = True )

In [ ]:
test_label

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features2/'
train_label.to_csv(outputpath + "train_action1.csv", index=False)
test_label.to_csv(outputpath + "test_action1.csv", index=False)